In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.marblenet as marblenet
import tensorflow as tf


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
config = malaya_speech.config.ctc_featurizer_config
config['feature_type'] = 'mfcc'
config['num_feature_bins'] = 64
featurizer = malaya_speech.tf_featurization.STTFeaturizer(**config)

In [5]:
i = tf.placeholder(tf.float32, [None])
v = featurizer.vectorize(i)
length_v = tf.cast(tf.shape(v)[0], tf.int32)
length_v = tf.expand_dims(length_v, 0)
v = tf.expand_dims(v, 0)
v, length_v

(<tf.Tensor 'ExpandDims_2:0' shape=(1, ?, 64) dtype=float32>,
 <tf.Tensor 'ExpandDims_1:0' shape=(1,) dtype=int32>)

In [6]:
model = marblenet.Model(v, length_v, factor = 5)
model.logits


Instructions for updating:
Use `tf.keras.layers.SeparableConv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.


{'outputs': <tf.Tensor 'w2l_encoder/Relu_8:0' shape=(1, ?, 640) dtype=float32>,
 'src_length': <tf.Tensor 'w2l_encoder/floordiv_16:0' shape=(1,) dtype=int32>}

In [7]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [8]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [9]:
l = sess.run(model.logits, feed_dict = {i: y})
l

{'outputs': array([[[1.6790601 , 0.71341133, 0.49537218, ..., 0.3699322 ,
          0.        , 0.5430219 ],
         [0.7126331 , 0.42140076, 1.5368685 , ..., 0.        ,
          0.        , 1.2523274 ],
         [0.8218622 , 1.4272544 , 0.0574941 , ..., 0.32133526,
          0.        , 0.02983855],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.47231466, 0.20830756, ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.0867331 ,
          0.        , 0.        ]]], dtype=float32),
 'src_length': array([561], dtype=int32)}

In [10]:
%%time

l = sess.run(model.logits, feed_dict = {i: y1})
l

CPU times: user 71.7 ms, sys: 12.2 ms, total: 83.9 ms
Wall time: 24.8 ms


{'outputs': array([[[1.2540596 , 1.0161332 , 0.        , ..., 1.8098981 ,
          0.00976402, 1.6998949 ],
         [0.66579807, 0.4262313 , 0.38872725, ..., 0.62833124,
          0.        , 0.9215847 ],
         [0.9172699 , 2.276891  , 0.        , ..., 0.15268469,
          0.99495655, 1.2799648 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.55098486, 0.169615  , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.65456444, 0.        , ..., 1.181891  ,
          0.        , 0.        ]]], dtype=float32),
 'src_length': array([350], dtype=int32)}

In [11]:
saver = tf.train.Saver()
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [12]:
!ls -lha test

total 16672
drwxr-xr-x    6 huseinzolkepli  staff   192B Jan  2 19:25 .
drwxr-xr-x  122 huseinzolkepli  staff   3.8K Jan  2 19:25 ..
-rw-r--r--    1 huseinzolkepli  staff    77B Jan  2 19:25 checkpoint
-rw-r--r--    1 huseinzolkepli  staff   7.8M Jan  2 19:25 model.ckpt.data-00000-of-00001
-rw-r--r--    1 huseinzolkepli  staff   3.1K Jan  2 19:25 model.ckpt.index
-rw-r--r--    1 huseinzolkepli  staff   304K Jan  2 19:25 model.ckpt.meta


In [13]:
!rm -rf test